In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt


# Possible augmentations : 8 isometries of the square

In this competition, resize or deform images is a bad idea, because of loss of information about steganography. So the ways to do image augmentation are limited.

However it's possible to do rotations and flips, by randomly transform images with one of the 8 isometries of the square. Those isometries are elements of dihedral group D4 (https://en.wikipedia.org/wiki/Examples_of_groups#dihedral_group_of_order_8).


In [ ]:
img = cv2.imread('/kaggle/input/alaska2-image-steganalysis/Cover/00002.jpg')
img = img[:,:,::-1]

fig, axs = plt.subplots(2,4, figsize=(12,6))

for i in range(8):
    ax = axs[i//4, i%4]
    im = img.transpose((i//4,1-i//4,2)) # optionnal rotation
    if i%2 == 1:
        im = im[::-1,:,:] # optionnal vertical flip
    if (i//2)%2 == 1:
        im = im[:,::-1,:] # optionnal horizontal flip
    
    ax.set_title(f'isometry_{i}')
    ax.imshow(im)
    ax.axis('off')

plt.show()


# Loss and competition metric results by isometry

The aim of this little kernel is to check if TTA is usefull.

To perform TTA with one image, we take the mean of probabilities (given by the CNN) of the 8 isometries of this image.

Once the model is trained, we realise TTA on a validation set of 15k images. Each isometry of each image is present once in the validation set, so we have 15k x 8 = 120k in our validation set.

In the following tab we present the results for each isometry and for the mean of the 8 isometries.




In [ ]:
df_results_by_iso = pd.read_csv('/kaggle/input/resultsbyiso/results_by_iso.csv')
df_results_iso_0 = pd.read_csv('/kaggle/input/resultsiso0/results_iso_0.csv').rename(columns={'compression JPEG':'compression_JPEG'})
df_results_iso_2 = pd.read_csv('/kaggle/input/results-iso-2/results_iso_2.csv')


# errors in dataframe names :

mask = df_results_by_iso.isometry == 'mean of 8 Iso_0'
df_results_by_iso.loc[mask, 'isometry'] = 'mean of 8 isometries'

for i in range(8):
    mask = df_results_by_iso.isometry == str(i)
    df_results_by_iso.loc[mask, 'isometry'] = f'iso_{i}'
    
df_plot = pd.concat([df_results_by_iso.iloc[-9:,:], df_results_iso_0.iloc[-1:,:], df_results_iso_2.iloc[-1:,:]])

In [ ]:
mask = df_results_by_iso.compression_JPEG == 'all'
df_results_by_iso[mask].iloc[:,1:]

# Comparing the results with 8 repetitions of the same isometry

It seems that TTA is very usefull, indeed the competition metric is jumping from approximately 0.912 to 0.923 !

However, since my pytorch CNN is evaluating images not with option "model.eval()" but "model.train()", the predictions are not deterministic. So the improvement with TTA can be due to the variability of predictions. In this case repeating the validation several times and take the mean of probabilities is a good way to improve the score.

To ensure that TTA is still usefull, a good way is to repeat 8 predictions of the same isometry, take the mean of probabilities and compare the results with TTA. Here we are doing this for both "iso 0" and "iso 2", always on the 15k validation set :

In [ ]:
df = pd.concat([df_results_iso_0, df_results_iso_2])
mask = df.compression_JPEG == 'all'
df[mask].iloc[:,1:]

By repeating 8 times the evaluation of each image and taking the mean we reach 0.919, against 0.923 with TTA.
We can conclude than TTA is usefull, gaining a 0.004 competition metric in our case.

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.bar(range(11), df_plot.weighted_AUC, width=0.7)

ax.set_xticks(range(11))
ax.set_xticklabels(df_plot.isometry, rotation=50)

plt.title('Competition metric scores : one isometry vs TTA')
plt.ylim([0.91, 0.924])
plt.ylabel('weighted AUC')

plt.show()